In [17]:
from threading import Thread
import serial
import time
import collections
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import struct
import copy
import pandas as pd

In [18]:
arduinoData = serial.Serial('COM4',115200)
time.sleep(1)
while(1==1):
    while(arduinoData.inWaiting()==0):
        pass
    dataPacket = arduinoData.readline()
    print(dataPacket)


b'9,-271,-1086,\r\n'
b'10,-275,-1087,\r\n'
b'10,-272,-1086,\r\n'
b'8,-272,-1080,\r\n'
b'8,-268,-1080,\r\n'
b'7,-267,-1078,\r\n'
b'7,-267,-1080,\r\n'
b'11,-266,-1081,\r\n'
b'10,-264,-1078,\r\n'
b'9,-265,-1079,\r\n'
b'13,-266,-1075,\r\n'
b'10,-265,-1071,\r\n'
b'11,-266,-1074,\r\n'
b'8,-264,-1081,\r\n'
b'6,-267,-1080,\r\n'
b'6,-269,-1080,\r\n'
b'7,-269,-1085,\r\n'
b'6,-270,-1085,\r\n'
b'7,-270,-1084,\r\n'
b'7,-269,-1087,\r\n'
b'5,-269,-1082,\r\n'
b'5,-267,-1075,\r\n'
b'4,-267,-1081,\r\n'
b'2,-269,-1082,\r\n'
b'7,-271,-1089,\r\n'
b'8,-271,-1084,\r\n'
b'5,-273,-1081,\r\n'
b'4,-272,-1084,\r\n'
b'5,-275,-1088,\r\n'
b'6,-276,-1088,\r\n'
b'7,-276,-1087,\r\n'
b'6,-275,-1088,\r\n'
b'6,-277,-1090,\r\n'
b'5,-274,-1089,\r\n'
b'2,-276,-1082,\r\n'
b'-1,-276,-1086,\r\n'
b'0,-276,-1089,\r\n'
b'1,-278,-1084,\r\n'
b'0,-280,-1087,\r\n'
b'1,-280,-1084,\r\n'
b'3,-280,-1085,\r\n'
b'4,-280,-1083,\r\n'
b'4,-277,-1085,\r\n'
b'2,-278,-1082,\r\n'
b'4,-275,-1084,\r\n'
b'7,-275,-1086,\r\n'
b'8,-277,-1085,\r\n'
b'5,-

b'6,-273,-1090,\r\n'
b'6,-273,-1086,\r\n'
b'4,-277,-1091,\r\n'
b'1,-278,-1090,\r\n'
b'0,-276,-1089,\r\n'
b'4,-277,-1085,\r\n'
b'3,-276,-1084,\r\n'
b'3,-275,-1085,\r\n'
b'0,-274,-1089,\r\n'
b'3,-270,-1084,\r\n'
b'3,-272,-1080,\r\n'
b'4,-271,-1083,\r\n'
b'5,-275,-1080,\r\n'
b'6,-274,-1078,\r\n'
b'6,-276,-1077,\r\n'
b'6,-273,-1075,\r\n'
b'6,-274,-1077,\r\n'
b'5,-274,-1077,\r\n'
b'6,-276,-1073,\r\n'
b'5,-274,-1076,\r\n'
b'6,-275,-1077,\r\n'
b'3,-271,-1084,\r\n'
b'4,-273,-1081,\r\n'
b'5,-276,-1079,\r\n'
b'7,-279,-1079,\r\n'
b'3,-275,-1078,\r\n'
b'3,-271,-1080,\r\n'
b'4,-276,-1081,\r\n'
b'7,-279,-1082,\r\n'
b'6,-278,-1079,\r\n'
b'4,-274,-1079,\r\n'
b'4,-275,-1080,\r\n'
b'3,-276,-1080,\r\n'
b'4,-277,-1088,\r\n'
b'4,-271,-1084,\r\n'
b'4,-272,-1083,\r\n'
b'4,-274,-1081,\r\n'
b'4,-275,-1082,\r\n'
b'4,-274,-1084,\r\n'
b'3,-272,-1085,\r\n'
b'6,-271,-1083,\r\n'
b'7,-271,-1090,\r\n'
b'9,-273,-1088,\r\n'
b'8,-273,-1090,\r\n'
b'8,-272,-1087,\r\n'
b'6,-274,-1083,\r\n'
b'7,-276,-1086,\r\n'
b'6,-276,-108

KeyboardInterrupt: 

In [19]:
arduinoData.close()

In [ ]:
class serialPlot:
    def __init__(self, serialPort, serialBaud=115200, plotLength=100, dataNumBytes=2, numPlots=1):
        self.port = serialPort
        self.baud = serialBaud
        self.plotMaxLength = plotLength
        self.dataNumBytes = dataNumBytes
        self.numPlots = numPlots
        self.rawData = bytearray(numPlots * dataNumBytes)
        self.dataType = None
        if dataNumBytes == 2:
            self.dataType = 'h'     # 2 byte integer
        elif dataNumBytes == 4:
            self.dataType = 'f'     # 4 byte float
        self.data = []
        for i in range(numPlots):   # give an array for each type of data and store them in a list
            self.data.append(collections.deque([0] * plotLength, maxlen=plotLength))
        self.isRun = True
        self.isReceiving = False
        self.thread = None
        self.plotTimer = 0
        self.previousTimer = 0
        # self.csvData = []

        print('Trying to connect to: ' + str(serialPort) + ' at ' + str(serialBaud) + ' BAUD.')
        try:
            self.serialConnection = serial.Serial(serialPort, serialBaud, timeout=4)
            print('Connected to ' + str(serialPort) + ' at ' + str(serialBaud) + ' BAUD.')
        except:
            print("Failed to connect with " + str(serialPort) + ' at ' + str(serialBaud) + ' BAUD.')
            
    def readSerialStart(self):
        if self.thread == None:
            self.thread = Thread(target=self.backgroundThread)
            self.thread.start()
            # Block till we start receiving values
            while self.isReceiving != True:
                time.sleep(0.1)
    
    def getSerialData(self, frame, lines, lineValueText, lineLabel, timeText):
        currentTimer = time.perf_counter()
        self.plotTimer = int((currentTimer - self.previousTimer) * 1000)     # the first reading will be erroneous
        self.previousTimer = currentTimer
        timeText.set_text('Plot Interval = ' + str(self.plotTimer) + 'ms')
        privateData = copy.deepcopy(self.rawData[:])    # so that the 3 values in our plots will be synchronized to the same sample time
        for i in range(self.numPlots):
            data = privateData[(i*self.dataNumBytes):(self.dataNumBytes + i*self.dataNumBytes)]
            value,  = struct.unpack(self.dataType, data)
            self.data[i].append(value)    # we get the latest data point and append it to our array
            lines[i].set_data(range(self.plotMaxLength), self.data[i])
            lineValueText[i].set_text('[' + lineLabel[i] + '] = ' + str(value))
        # self.csvData.append([self.data[0][-1], self.data[1][-1], self.data[2][-1]])

    def backgroundThread(self):    # retrieve data
        time.sleep(1.0)  # give some buffer time for retrieving data
        self.serialConnection.reset_input_buffer()
        while (self.isRun):
            self.serialConnection.readinto(self.rawData)
            self.isReceiving = True
            #print(self.rawData)

    def close(self):
        self.isRun = False
        self.thread.join()
        self.serialConnection.close()
        print('Disconnected...')
        # df = pd.DataFrame(self.csvData)
        # df.to_csv('/home/rikisenia/Desktop/data.csv')


In [16]:
def main():
    portName = 'COM4'
    # portName = '/dev/ttyUSB0'
    baudRate = 115200
    maxPlotLength = 100     # number of points in x-axis of real time plot
    dataNumBytes = 2        # number of bytes of 1 data point
    numPlots = 3            # number of plots in 1 graph
    s = serialPlot(portName, baudRate, maxPlotLength, dataNumBytes, numPlots)   # initializes all required variables
    s.readSerialStart()                                               # starts background thread
    print('Ready3')
    
    # plotting starts below
    pltInterval = 50    # Period at which the plot animation updates [ms]
    xmin = 0
    xmax = maxPlotLength
    ymin = -(1)
    ymax = 1
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(xlim=(xmin, xmax), ylim=(float(ymin - (ymax - ymin) / 10), float(ymax + (ymax - ymin) / 10)))
    ax.set_title('Arduino Accelerometer')
    ax.set_xlabel("Time")
    ax.set_ylabel("Accelerometer Output")

    lineLabel = ['X', 'Y', 'Z']
    style = ['r-', 'c-', 'b-']  # linestyles for the different plots
    timeText = ax.text(0.70, 0.95, '', transform=ax.transAxes)
    lines = []
    lineValueText = []
    for i in range(numPlots):
        lines.append(ax.plot([], [], style[i], label=lineLabel[i])[0])
        lineValueText.append(ax.text(0.70, 0.90-i*0.05, '', transform=ax.transAxes))
    anim = animation.FuncAnimation(fig, s.getSerialData, fargs=(lines, lineValueText, lineLabel, timeText), interval=pltInterval)    # fargs has to be a tuple

    plt.legend(loc="upper left")
    plt.show()

    s.close()


if __name__ == '__main__':
    main()

Trying to connect to: COM4 at 115200 BAUD.
Failed to connect with COM4 at 115200 BAUD.


Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\Users\Ram\anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\Ram\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-14-acf766c5d301>", line 55, in backgroundThread
    self.serialConnection.reset_input_buffer()
AttributeError: 'serialPlot' object has no attribute 'serialConnection'



KeyboardInterrupt: 